In [ ]:
import sys
sys.path.append('../')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from configs import base_config
from model import T2T_ViT
from utils import Trainer, WarmupCosineSchedule, WarmupLinearSchedule, build_model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
learning_rate = 1e-3
batch_size = 64
num_epochs = 20

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='../datasets', train=True, transform=transform)
val_dataset = datasets.CIFAR10(root='../datasets', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model_config = base_config()
model_config.update({'num_classes': 10})
model = build_model(T2T_ViT, model_config).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = WarmupCosineSchedule(optimizer, num_epochs//5, num_epochs)
criterion = nn.CrossEntropyLoss()

In [ ]:
trainer = Trainer(model, {'train':train_loader, 'validation':val_loader}, criterion, optimizer, scheduler, num_epochs, (1, ), 'pretrained/cifar10_224.pth', device)

In [ ]:
trainer.train()